In [33]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

In [19]:
#Data loading 
loader = PyPDFLoader("../data/Ebook-Agentic-AI.pdf")
documents = loader.load()

In [21]:
#Chunking 
splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=150
)

chunks = splitter.split_documents(documents)
print(f"Total chunks: {len(chunks)}")

Total chunks: 126


In [ ]:
#Embedding
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

C:\Users\ACER\AppData\Local\Temp\ipykernel_4712\1025830397.py:12: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [24]:
#Vector Store
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="../chroma_db"
)

vectorstore.persist()

In [32]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}
)

In [47]:
api_key = os.getenv("Groq_API")
llm = ChatGroq(
    model_name="llama-3.1-8b-instant",
    temperature=0,
    api_key=api_key
)

In [48]:
#RAG Prompt
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
You are a retrieval-augmented AI.
Answer ONLY using the context below.
If the answer is not in the context, say:
"I cannot answer based on the provided document."

Context:
{context}

Question:
{question}
""")


In [49]:
#Executing RAG
def rag_answer(question):
    docs_and_scores = vectorstore.similarity_search_with_score(question, k=4)

    docs = [d for d, _ in docs_and_scores]
    scores = [s for _, s in docs_and_scores]

    context = "\n\n".join(d.page_content for d in docs)

    response = llm.invoke(
        prompt.format(context=context, question=question)
    )

    confidence = round(1 - sum(scores)/len(scores), 3)

    return {
        "answer": response.content,
        "context": context,
        "confidence": confidence
    }
rag_answer("What is Agentic AI?")

{'answer': 'In this section, we will define what Agentic AI is and, more importantly, what it’s not, as it’s often misunderstood. \n\nAgentic AI promises a shift from reactive to proactive problem-solving.',
 'context': "Agentic AI \nAn Executive's Guide to In-depth \nUnderstanding of Agentic AI\n\nAgentic AI \nAn Executive's Guide to In-depth \nUnderstanding of Agentic AI\n\nINTRODUCTION \nTO AGENTIC AI \n \n \n \nIn this section, we will define what Agentic AI is and, more importantly, \nwhat it’s not, as it’s often misunderstood. While Agentic AI promises a \nshift from reactive to proactive problem-solving, doubts remain. Is it truly \nreal? Are all the claims about it accurate? \nIn this section, we cover the following topics: \n● What is Agentic AI? \n● How does it stand apart from other AI ,What can it do? \n● What value does it bring? \n● How are businesses using it in the real world? \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \

In [50]:
rag_answer("What is Multi Agent?")

{'answer': 'Multi-Agent Systems (MAS) refer to a category of agentic systems that can leverage feedback from other agents to verify their work and improve task outcomes. They are particularly effective in tasks requiring diverse feedback and parallel task execution.',
 'context': "AGENTIC  AI FOR EXECUTIVES \n24 \n \n \n \nWhat Are Multi-Agent Systems? \nAgentic systems can be categorized as single and multi-agent systems (MAS). \nSingle-agent architectures are efficient in scenarios where tasks are well-defined, \nand processes are systematic. Such architectures benefit from a single agent's ability \nto operate without the risk of interference from other agents' feedback, which can be \ndistracting. However, they may hit execution loops if their reasoning capabilities \naren't robust. \n \nIn contrast, MAS shines in tasks requiring diverse feedback and parallel task \nexecution. These systems can leverage feedback from other agents to verify their \nwork and improve task outcomes. \n